# Filtros, Cartonize

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def cartoonize(rgb_image: np.ndarray, num_pyr_downs=2, num_bilaterals=7):
    # STEP 1 -- Apply a bilateral filter to reduce the color palette of
    # the image.
    downsampled_img = rgb_image
    for _ in range(num_pyr_downs):
        downsampled_img = cv2.pyrDown(downsampled_img)

    for _ in range(num_bilaterals):
        filterd_small_img = cv2.bilateralFilter(downsampled_img, 9, 9, 7)

    filtered_normal_img = filterd_small_img
    for _ in range(num_pyr_downs):
        filtered_normal_img = cv2.pyrUp(filtered_normal_img)

    # make sure resulting image has the same dims as original
    if filtered_normal_img.shape != rgb_image.shape:
        filtered_normal_img = cv2.resize(
            filtered_normal_img, rgb_image.shape[:2])

    # STEP 2 -- Convert the original color image into grayscale.
    img_gray = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    # STEP 3 -- Apply a median blur to reduce image noise.
    img_blur = cv2.medianBlur(img_gray, 7)

    # STEP 4 -- Use adaptive thresholding to detect and emphasize the edges
    # in an edge mask.
    gray_edges = cv2.adaptiveThreshold(img_blur, 255,
                                       cv2.ADAPTIVE_THRESH_MEAN_C,
                                       cv2.THRESH_BINARY, 9, 2)
    # STEP 5 -- Combine the color image from step 1 with the edge mask
    # from step 4.
    rgb_edges = cv2.cvtColor(gray_edges, cv2.COLOR_GRAY2RGB)
    cartoon_image = cv2.bitwise_and(filtered_normal_img, rgb_edges)
   
                   
    return cartoon_image

In [9]:
# Definir la camara a utilizar
capture = cv2.VideoCapture(0)

# Obtener ciertas propiedades del dispositivo de captura (frame width, frame height y frames per second (fps)):
frame_width = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = capture.get(cv2.CAP_PROP_FPS)

# Imprimir estos valores
print("CV_CAP_PROP_FRAME_WIDTH: '{}'".format(frame_width))
print("CV_CAP_PROP_FRAME_HEIGHT : '{}'".format(frame_height))
print("CAP_PROP_FPS : '{}'".format(fps))

# Verificar si la camara se abrio adecuadamente
if capture.isOpened() is False:
    print("Error al abrir la camara")

CV_CAP_PROP_FRAME_WIDTH: '640.0'
CV_CAP_PROP_FRAME_HEIGHT : '480.0'
CAP_PROP_FPS : '30.0'


In [10]:
# Contador para guardar las imágenes
frame_index = 0
factor = 1
# Leer hasta que se completen las acciones
while capture.isOpened():
    # Capturar frame-by-frame la información de la camara
    ret, frame = capture.read()

    if ret is True:
        # Mostrar el frame capturado
        
        # Nuevo Tamaño
        h,w,c  = frame.shape
        # resize image
        #frame = cv2.resize(frame, (int(w*factor), int(h*factor)))
             
        imgCartoon = cartoonize(frame)
        cv2.imshow('Resultado', imgCartoon)
        
        
        # Presionar C para guardar
        if cv2.waitKey(20) & 0xFF == ord('c'):
            frame_name = "out/camera_frame_{}.png".format(frame_index)
            cv2.imwrite(frame_name, imgCartoon)
            
            frame_index += 1

        # Presionar q para salir
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    # Romper el bucle
    else:
        break

# Liberar camara y limpiar ventanas
capture.release()
cv2.destroyAllWindows()
